In [1]:
import pandas as pd
from pybedtools import BedTool

# 查看ATAC-seq在目标位置的信号变化

## 准备df

### df_ctcf
- weak
- medium
- strong
- overlapped with offs

In [2]:
df_new_offs = pd.read_csv(
    '/lustre1/chengqiyi_pkuhpc/zhaohn/3.project/2022_DdCBE-3D-Genome_topic/'
    '2022-09-30_Detect-seq_batch-1_ATP8_JAK2_SIRT6/final_list_after_igv_check/'
    '2023-04-24_merged_final_list_add_header_poisson_result.csv',
    header=0, 
)
df_new_offs = df_new_offs[['chr_name', 'region_start', 'region_end']].copy()
df_new_offs.columns = ['chrom', 'start', 'end']
df_new_offs.drop_duplicates(keep='first', inplace=True)
df_new_offs

,chrom,start,end
0,chr11,28107123,28107196
1,chr4,127523472,127523528
2,chr10,79996559,79996604
3,chr20,36808712,36808741
4,chr9,115952242,115952303
...,...,...,...
15392,chr1,148725246,148725266
15393,chr8,81158725,81158758
15394,chr8,100191816,100191830
15395,chr17,32831978,32831991


In [3]:
df_old_offs = pd.read_csv(
    '/lustre1/chengqiyi_pkuhpc/zhaohn/3.project/2022_DdCBE-3D-Genome_topic/'
    '2022-09-30_Detect-seq_batch-1_ATP8_JAK2_SIRT6/tables/'
    '2022-10-05_WT_ND4-5.1-6_final-list.bed',
    header=None, sep='\t', names=['chrom', 'start', 'end', 'name', 'score', 'strand']
)
df_old_offs = df_old_offs.query('name.str.contains("IND")').copy()
df_old_offs

,chrom,start,end,name,score,strand
0,chr1,2044988,2044998,ND5.1-TAS.IND-1_chr1_2044988_2044998,7,+
1,chr1,9702414,9702449,ND5.1-TAS.IND-2_ND6-TAS.IND-1_chr1_9702414_970...,11,-
2,chr1,12618165,12618174,ND5.1-TAS.IND-3_chr1_12618165_12618174,5,+
3,chr1,13019998,13020019,ND6-TAS.IND-2_chr1_13019998_13020019,9,-
4,chr1,13354705,13354738,ND6-TAS.IND-3_chr1_13354705_13354738,11,+
...,...,...,...,...,...,...
1027,chrX,136925330,136925338,ND6-TAS.IND-538_chrX_136925330_136925338,5,+
1028,chrX,138128211,138128237,ND6-TAS.IND-539_chrX_138128211_138128237,9,-
1029,chrX,150683735,150683758,ND5.1-TAS.IND-452_ND6-TAS.IND-540_chrX_1506837...,11,-
1032,chrX,153669436,153669453,ND5.1-TAS.IND-453_ND6-TAS.IND-541_chrX_1536694...,10,+


In [4]:
df_offs = pd.concat(
    [
        df_old_offs.iloc[:,:3],
        df_new_offs.iloc[:,:3]
    ]
)
df_offs

,chrom,start,end
0,chr1,2044988,2044998
1,chr1,9702414,9702449
2,chr1,12618165,12618174
3,chr1,13019998,13020019
4,chr1,13354705,13354738
...,...,...,...
15392,chr1,148725246,148725266
15393,chr8,81158725,81158758
15394,chr8,100191816,100191830
15395,chr17,32831978,32831991


In [5]:
df_ctcf = pd.read_csv('/lustre1/chengqiyi_pkuhpc/zhaohn/1.database/public_data/CTCF_ATAC_data/ENCFF285QVL_CTCF_binding_sites_fix_range.bed',
                     sep='\t')
df_ctcf

,chrom,start,end,name,RPKM,strand
0,chr1,267790,268214,chr1_267790_268214_weak_peak,1.568319,.
1,chr1,778701,779047,chr1_778701_779047_strong_peak,5.326924,.
2,chr1,778863,779287,chr1_778863_779287_medium_peak,2.778652,.
3,chr1,869675,870099,chr1_869675_870099_weak_peak,1.125098,.
4,chr1,904753,904883,chr1_904753_904883_strong_peak,6.115919,.
...,...,...,...,...,...,...
38030,chrX,155264125,155264549,chrX_155264125_155264549_medium_peak,3.204826,.
38031,chrX,155686260,155686684,chrX_155686260_155686684_weak_peak,1.295568,.
38032,chrX,155820126,155820550,chrX_155820126_155820550_medium_peak,3.426436,.
38033,chrX,155881180,155881604,chrX_155881180_155881604_medium_peak,3.068450,.


In [6]:
a = BedTool.from_dataframe(df_offs.iloc[:,:3])
b = BedTool.from_dataframe(df_ctcf.iloc[:,:3])
# 去掉在off里的ctcf
df_ctcf_out_offs = b.intersect(a, loj=True).to_dataframe().query('score==-1').iloc[:, :3].merge(df_ctcf, on=['chrom', 'start', 'end'], how='left')
df_ctcf_in_offs = b.intersect(a, loj=True).to_dataframe().query('score!=-1').iloc[:, :3].merge(df_ctcf, on=['chrom', 'start', 'end'], how='left')
df_ctcf_out_offs

,chrom,start,end,name,RPKM,strand
0,chr1,267790,268214,chr1_267790_268214_weak_peak,1.568319,.
1,chr1,778701,779047,chr1_778701_779047_strong_peak,5.326924,.
2,chr1,778863,779287,chr1_778863_779287_medium_peak,2.778652,.
3,chr1,869675,870099,chr1_869675_870099_weak_peak,1.125098,.
4,chr1,904753,904883,chr1_904753_904883_strong_peak,6.115919,.
...,...,...,...,...,...,...
36777,chrX,155264125,155264549,chrX_155264125_155264549_medium_peak,3.204826,.
36778,chrX,155686260,155686684,chrX_155686260_155686684_weak_peak,1.295568,.
36779,chrX,155820126,155820550,chrX_155820126_155820550_medium_peak,3.426436,.
36780,chrX,155881180,155881604,chrX_155881180_155881604_medium_peak,3.068450,.


In [7]:
df_ctcf_out_offs['signal_strength'] = df_ctcf_out_offs['name'].str.split('_').str[-2]
df_ctcf_out_offs

,chrom,start,end,name,RPKM,strand,signal_strength
0,chr1,267790,268214,chr1_267790_268214_weak_peak,1.568319,.,weak
1,chr1,778701,779047,chr1_778701_779047_strong_peak,5.326924,.,strong
2,chr1,778863,779287,chr1_778863_779287_medium_peak,2.778652,.,medium
3,chr1,869675,870099,chr1_869675_870099_weak_peak,1.125098,.,weak
4,chr1,904753,904883,chr1_904753_904883_strong_peak,6.115919,.,strong
...,...,...,...,...,...,...,...
36777,chrX,155264125,155264549,chrX_155264125_155264549_medium_peak,3.204826,.,medium
36778,chrX,155686260,155686684,chrX_155686260_155686684_weak_peak,1.295568,.,weak
36779,chrX,155820126,155820550,chrX_155820126_155820550_medium_peak,3.426436,.,medium
36780,chrX,155881180,155881604,chrX_155881180_155881604_medium_peak,3.068450,.,medium


In [8]:
df_ctcf_in_offs['signal_strength'] = df_ctcf_in_offs['name'].str.split('_').str[-2]
df_ctcf_in_offs

,chrom,start,end,name,RPKM,strand,signal_strength
0,chr1,2044897,2045321,chr1_2044897_2045321_medium_peak,3.477577,.,medium
1,chr1,2044897,2045321,chr1_2044897_2045321_medium_peak,3.477577,.,medium
2,chr1,2139943,2140169,chr1_2139943_2140169_strong_peak,8.826999,.,strong
3,chr1,9428572,9428996,chr1_9428572_9428996_strong_peak,4.704957,.,strong
4,chr1,9702095,9702519,chr1_9702095_9702519_weak_peak,1.892211,.,weak
...,...,...,...,...,...,...,...
1575,chrX,153922850,153923274,chrX_153922850_153923274_strong_peak,3.682140,.,strong
1576,chrX,154013581,154014005,chrX_154013581_154014005_strong_peak,4.261736,.,strong
1577,chrX,154980943,154981142,chrX_154980943_154981142_strong_peak,7.627437,.,strong
1578,chrX,155048969,155049393,chrX_155048969_155049393_medium_peak,2.574089,.,medium


In [9]:
df_ctcf_out_offs.signal_strength.value_counts()

signal_strength
strong    12473
medium    12166
weak      12143
Name: count, dtype: int64

In [10]:
df_ctcf_in_offs.signal_strength.value_counts()

signal_strength
strong    663
medium    520
weak      397
Name: count, dtype: int64

In [11]:
36565 / (110 + 107 + 36565) * 1580

# 所以从CTCF强度上看不出有无offtarget的差别

1570.6785927899516

In [12]:
for signal, df in df_ctcf_out_offs.groupby('signal_strength'):
    # print(signal)
    # print(df.drop(columns='signal_strength'))
    df.drop(columns='signal_strength').to_csv(f'../bed_for_count/2023-05-12_CTCF_{signal}_for_count.bed', header=False, index=False, sep='\t')

In [13]:
df_ctcf_in_offs.drop(columns='signal_strength').to_csv(f'../bed_for_count/2023-05-12_CTCF_within-offs_for_count.bed', header=False, index=False, sep='\t')

### df_offs

In [14]:
df_offs = pd.read_csv(
    '/lustre1/chengqiyi_pkuhpc/zhaohn/3.project/2022_DdCBE-3D-Genome_topic/'
    '2022-09-30_Detect-seq_batch-1_ATP8_JAK2_SIRT6/tables/'
    '2022-10-05_WT_ND4-5.1-6_final-list.bed',
    header=None, sep='\t', names=['chrom', 'start', 'end', 'name', 'score', 'strand']
)
df_offs

,chrom,start,end,name,score,strand
0,chr1,2044988,2044998,ND5.1-TAS.IND-1_chr1_2044988_2044998,7,+
1,chr1,9702414,9702449,ND5.1-TAS.IND-2_ND6-TAS.IND-1_chr1_9702414_970...,11,-
2,chr1,12618165,12618174,ND5.1-TAS.IND-3_chr1_12618165_12618174,5,+
3,chr1,13019998,13020019,ND6-TAS.IND-2_chr1_13019998_13020019,9,-
4,chr1,13354705,13354738,ND6-TAS.IND-3_chr1_13354705_13354738,11,+
...,...,...,...,...,...,...
1030,chrX,150772346,150772356,ND6-Extend-38_chrX_150772346_150772356,6,-
1031,chrX,151608905,151608918,ND5.1-TAS.DR-91_chrX_151608905_151608918,7,+
1032,chrX,153669436,153669453,ND5.1-TAS.IND-453_ND6-TAS.IND-541_chrX_1536694...,10,+
1033,chrX,154373542,154373556,ND5.1-Extend-75_chrX_154373542_154373556,6,-


In [15]:
df_offs_IND = pd.concat(
    [
        # df_offs.query('name.str.contains("ND6-TAS.DL")'),
        # df_offs.query('name.str.contains("ND6-TAS.DR")'),
        df_offs.query('name.str.contains("ND6-TAS.IND")')
    ]
)
df_offs_IND

,chrom,start,end,name,score,strand
1,chr1,9702414,9702449,ND5.1-TAS.IND-2_ND6-TAS.IND-1_chr1_9702414_970...,11,-
3,chr1,13019998,13020019,ND6-TAS.IND-2_chr1_13019998_13020019,9,-
4,chr1,13354705,13354738,ND6-TAS.IND-3_chr1_13354705_13354738,11,+
5,chr1,15046702,15046715,ND6-TAS.IND-4_chr1_15046702_15046715,7,+
6,chr1,16082505,16082527,ND5.1-TAS.IND-4_ND6-TAS.IND-5_chr1_16082505_16...,10,-
...,...,...,...,...,...,...
1027,chrX,136925330,136925338,ND6-TAS.IND-538_chrX_136925330_136925338,5,+
1028,chrX,138128211,138128237,ND6-TAS.IND-539_chrX_138128211_138128237,9,-
1029,chrX,150683735,150683758,ND5.1-TAS.IND-452_ND6-TAS.IND-540_chrX_1506837...,11,-
1032,chrX,153669436,153669453,ND5.1-TAS.IND-453_ND6-TAS.IND-541_chrX_1536694...,10,+


In [16]:
df_offs_DEP = pd.concat(
    [
        df_offs.query('name.str.contains("ND6-TAS.DL")'),
        df_offs.query('name.str.contains("ND6-TAS.DR")'),
        # df_offs.query('name.str.contains("ND6-TAS.IND")')
    ]
)
df_offs_DEP

,chrom,start,end,name,score,strand
17,chr1,31956473,31956495,ND6-TAS.DL-1_chr1_31956473_31956495,10,+
23,chr1,35079561,35079576,ND6-TAS.DL-2_chr1_35079561_35079576,8,-
36,chr1,54578134,54578149,ND6-TAS.DL-3_chr1_54578134_54578149,3,-
37,chr1,54671922,54671938,ND6-TAS.DL-4_chr1_54671922_54671938,13,-
65,chr1,95036181,95036212,ND6-TAS.DL-5_chr1_95036181_95036212,8,-
...,...,...,...,...,...,...
974,chr22,26135728,26135754,ND6-TAS.DR-29_chr22_26135728_26135754,7,-
983,chr22,35676615,35676639,ND6-TAS.DR-30_chr22_35676615_35676639,8,-
1003,chrX,25020353,25020385,ND6-TAS.DR-31_chrX_25020353_25020385,14,-
1016,chrX,106613768,106613787,ND6-TAS.DR-32_chrX_106613768_106613787,6,-


In [17]:
df_offs_IND.to_csv(f'../bed_for_count/2023-05-12_offs-ND6-IND_for_count.bed', header=False, index=False, sep='\t')
df_offs_DEP.to_csv(f'../bed_for_count/2023-05-12_offs-ND6-DEP_for_count.bed', header=False, index=False, sep='\t')

### df_random

In [18]:
random = BedTool().random(l=200, n=33000, genome='hg38', seed=123)
print(random.to_dataframe())
random_fa = random.sequence(fi='/lustre1/chengqiyi_pkuhpc/zhaohn/1.database/db_genomes/genome_fa/genome_ucsc_hg38/genome_ucsc_hg38.fa')
df_random = pd.DataFrame(random_fa.print_sequence().split('>')[1:], columns=['info'])
df_random[['coordinate', 'sequence']] = df_random['info'].str.strip().str.split('\n', expand=True)
df_random = df_random.query('~sequence.str.upper().str.contains("N")').copy()
df_random

       chrom      start        end   name  score strand
0       chr1  171273699  171273899      1    200      +
1      chr10  118814584  118814784      2    200      -
2      chr17   54532798   54532998      3    200      -
3       chr7   70496778   70496978      4    200      -
4       chr2    2558151    2558351      5    200      -
...      ...        ...        ...    ...    ...    ...
32995   chrX  100943277  100943477  32996    200      -
32996   chr5  102140158  102140358  32997    200      -
32997  chr16    2979521    2979721  32998    200      +
32998   chr3  141405823  141406023  32999    200      +
32999  chr20    7284852    7285052  33000    200      +

[33000 rows x 6 columns]


WARNING. chromosome (chr19_KI270930v1_alt) was not found in the FASTA file. Skipping.
WARNING. chromosome (chr6_GL000256v2_alt) was not found in the FASTA file. Skipping.
WARNING. chromosome (chr13_KI270843v1_alt) was not found in the FASTA file. Skipping.
WARNING. chromosome (chr16_KI270853v1_alt) was not found in the FASTA file. Skipping.
WARNING. chromosome (chr19_GL949746v1_alt) was not found in the FASTA file. Skipping.
WARNING. chromosome (chr17_KI270857v1_alt) was not found in the FASTA file. Skipping.
WARNING. chromosome (chr6_GL000253v2_alt) was not found in the FASTA file. Skipping.
WARNING. chromosome (chr17_GL000258v2_alt) was not found in the FASTA file. Skipping.
WARNING. chromosome (chr17_GL000258v2_alt) was not found in the FASTA file. Skipping.
WARNING. chromosome (chr16_KI270728v1_random) was not found in the FASTA file. Skipping.
WARNING. chromosome (chr15_KI270850v1_alt) was not found in the FASTA file. Skipping.
WARNING. chromosome (chr4_GL383528v1_alt) was not fou

,info,coordinate,sequence
0,chr1:171273699-171273899\nagagataaggtctcactatg...,chr1:171273699-171273899,agagataaggtctcactatgccgcccagggtggtcttgaactcctg...
1,chr10:118814584-118814784\ngtgagcccccttgcccaag...,chr10:118814584-118814784,gtgagcccccttgcccaaggtctcacctccttcttagagagcttgt...
2,chr17:54532798-54532998\ncaaatgatatttctgcctcta...,chr17:54532798-54532998,caaatgatatttctgcctctatgttttcttttgtttaattttgagt...
3,chr7:70496778-70496978\ntcagacacacacacacacccca...,chr7:70496778-70496978,tcagacacacacacacaccccacacatgcacacgtcacatcagcat...
4,chr2:2558151-2558351\nGAAGGTGAACAGACTGCCCTTCAA...,chr2:2558151-2558351,GAAGGTGAACAGACTGCCCTTCAAATCAAATTTGGAGTAATGGAGA...
...,...,...,...
31841,chrX:100943277-100943477\ncataataggatattgtttta...,chrX:100943277-100943477,cataataggatattgttttatgtttttatactttacgttaatacta...
31842,chr5:102140158-102140358\nCCATATGCACTCTTATACAG...,chr5:102140158-102140358,CCATATGCACTCTTATACAGAGAAAAGCCACCACATATCAAGCTAT...
31843,chr16:2979521-2979721\nAGGGTGTTGGGATCCTGGAGGTC...,chr16:2979521-2979721,AGGGTGTTGGGATCCTGGAGGTCACTGGTCACAAGCACAGCCTCCA...
31844,chr3:141405823-141406023\nCTCTGTTGAGGAAAGAAGAT...,chr3:141405823-141406023,CTCTGTTGAGGAAAGAAGATATGCAGAAATAACCACGTGATAAATG...


In [19]:
df_random[['chrom', 'start', 'end']] = df_random['coordinate'].str.replace(':', '\t').str.replace('-', '\t').str.split('\t', expand=True)
df_random[['start', 'end']] = df_random[['start', 'end']].astype(int)
df_random

,info,coordinate,sequence,chrom,start,end
0,chr1:171273699-171273899\nagagataaggtctcactatg...,chr1:171273699-171273899,agagataaggtctcactatgccgcccagggtggtcttgaactcctg...,chr1,171273699,171273899
1,chr10:118814584-118814784\ngtgagcccccttgcccaag...,chr10:118814584-118814784,gtgagcccccttgcccaaggtctcacctccttcttagagagcttgt...,chr10,118814584,118814784
2,chr17:54532798-54532998\ncaaatgatatttctgcctcta...,chr17:54532798-54532998,caaatgatatttctgcctctatgttttcttttgtttaattttgagt...,chr17,54532798,54532998
3,chr7:70496778-70496978\ntcagacacacacacacacccca...,chr7:70496778-70496978,tcagacacacacacacaccccacacatgcacacgtcacatcagcat...,chr7,70496778,70496978
4,chr2:2558151-2558351\nGAAGGTGAACAGACTGCCCTTCAA...,chr2:2558151-2558351,GAAGGTGAACAGACTGCCCTTCAAATCAAATTTGGAGTAATGGAGA...,chr2,2558151,2558351
...,...,...,...,...,...,...
31841,chrX:100943277-100943477\ncataataggatattgtttta...,chrX:100943277-100943477,cataataggatattgttttatgtttttatactttacgttaatacta...,chrX,100943277,100943477
31842,chr5:102140158-102140358\nCCATATGCACTCTTATACAG...,chr5:102140158-102140358,CCATATGCACTCTTATACAGAGAAAAGCCACCACATATCAAGCTAT...,chr5,102140158,102140358
31843,chr16:2979521-2979721\nAGGGTGTTGGGATCCTGGAGGTC...,chr16:2979521-2979721,AGGGTGTTGGGATCCTGGAGGTCACTGGTCACAAGCACAGCCTCCA...,chr16,2979521,2979721
31844,chr3:141405823-141406023\nCTCTGTTGAGGAAAGAAGAT...,chr3:141405823-141406023,CTCTGTTGAGGAAAGAAGATATGCAGAAATAACCACGTGATAAATG...,chr3,141405823,141406023


In [20]:
df_random = df_random.merge(random.to_dataframe(), on=['chrom', 'start', 'end'], how='left')
df_random

,info,coordinate,sequence,chrom,start,end,name,score,strand
0,chr1:171273699-171273899\nagagataaggtctcactatg...,chr1:171273699-171273899,agagataaggtctcactatgccgcccagggtggtcttgaactcctg...,chr1,171273699,171273899,1,200,+
1,chr10:118814584-118814784\ngtgagcccccttgcccaag...,chr10:118814584-118814784,gtgagcccccttgcccaaggtctcacctccttcttagagagcttgt...,chr10,118814584,118814784,2,200,-
2,chr17:54532798-54532998\ncaaatgatatttctgcctcta...,chr17:54532798-54532998,caaatgatatttctgcctctatgttttcttttgtttaattttgagt...,chr17,54532798,54532998,3,200,-
3,chr7:70496778-70496978\ntcagacacacacacacacccca...,chr7:70496778-70496978,tcagacacacacacacaccccacacatgcacacgtcacatcagcat...,chr7,70496778,70496978,4,200,-
4,chr2:2558151-2558351\nGAAGGTGAACAGACTGCCCTTCAA...,chr2:2558151-2558351,GAAGGTGAACAGACTGCCCTTCAAATCAAATTTGGAGTAATGGAGA...,chr2,2558151,2558351,5,200,-
...,...,...,...,...,...,...,...,...,...
30311,chrX:100943277-100943477\ncataataggatattgtttta...,chrX:100943277-100943477,cataataggatattgttttatgtttttatactttacgttaatacta...,chrX,100943277,100943477,32996,200,-
30312,chr5:102140158-102140358\nCCATATGCACTCTTATACAG...,chr5:102140158-102140358,CCATATGCACTCTTATACAGAGAAAAGCCACCACATATCAAGCTAT...,chr5,102140158,102140358,32997,200,-
30313,chr16:2979521-2979721\nAGGGTGTTGGGATCCTGGAGGTC...,chr16:2979521-2979721,AGGGTGTTGGGATCCTGGAGGTCACTGGTCACAAGCACAGCCTCCA...,chr16,2979521,2979721,32998,200,+
30314,chr3:141405823-141406023\nCTCTGTTGAGGAAAGAAGAT...,chr3:141405823-141406023,CTCTGTTGAGGAAAGAAGATATGCAGAAATAACCACGTGATAAATG...,chr3,141405823,141406023,32999,200,+


In [21]:
df_random = df_random[['chrom', 'start', 'end', 'name', 'score', 'strand']].copy()
df_random

,chrom,start,end,name,score,strand
0,chr1,171273699,171273899,1,200,+
1,chr10,118814584,118814784,2,200,-
2,chr17,54532798,54532998,3,200,-
3,chr7,70496778,70496978,4,200,-
4,chr2,2558151,2558351,5,200,-
...,...,...,...,...,...,...
30311,chrX,100943277,100943477,32996,200,-
30312,chr5,102140158,102140358,32997,200,-
30313,chr16,2979521,2979721,32998,200,+
30314,chr3,141405823,141406023,32999,200,+


In [22]:
# rm overlapped with df_ctcf, df_offs
a = BedTool.from_dataframe(df_ctcf.iloc[:,:3])
b = BedTool.from_dataframe(df_random.iloc[:,:3])
df_random = b.intersect(a, loj=True).to_dataframe().query('score==-1').iloc[:, :3].merge(df_random, on=['chrom', 'start', 'end'], how='left')



df_offs = pd.concat(
    [
        df_old_offs.iloc[:,:3],
        df_new_offs.iloc[:,:3]
    ]
)
# df_offs



a = BedTool.from_dataframe(df_offs.iloc[:,:3])
b = BedTool.from_dataframe(df_random.iloc[:,:3])
df_random = b.intersect(a, loj=True).to_dataframe().query('score==-1').iloc[:, :3].merge(df_random, on=['chrom', 'start', 'end'], how='left')


df_random

,chrom,start,end,name,score,strand
0,chr1,171273699,171273899,1,200,+
1,chr10,118814584,118814784,2,200,-
2,chr17,54532798,54532998,3,200,-
3,chr7,70496778,70496978,4,200,-
4,chr2,2558151,2558351,5,200,-
...,...,...,...,...,...,...
30065,chrX,100943277,100943477,32996,200,-
30066,chr5,102140158,102140358,32997,200,-
30067,chr16,2979521,2979721,32998,200,+
30068,chr3,141405823,141406023,32999,200,+


In [23]:
df_random.to_csv(f'../bed_for_count/2023-05-12_random_for_count.bed', header=False, index=False, sep='\t')

### df_tss

In [24]:
df_tss = pd.read_csv('ref_data/hg38_ucsc_refseq_NM_TSS_extend2kb.sorted.bed', sep='\t', header=None,
                    names=['chrom', 'start', 'end', 'name', 'score', 'strand'])
df_tss

,chrom,start,end,name,score,strand
0,chr1,63418,67418,NM_001005484,65418,+
1,chr1,449678,453678,NM_001005221,451678,-
2,chr1,684654,688654,NM_001005277,686654,-
3,chr1,921922,925922,NM_001385641,923922,+
4,chr1,923730,927730,NM_152486,925730,+
...,...,...,...,...,...,...
28009,chrY,24811393,24815393,NM_020364,24813393,-
28010,chrY,24831819,24835819,NM_020420,24833819,+
28011,chrY,24831918,24835918,NM_001388484,24833918,+
28012,chrY,25050104,25054104,NM_001002761,25052104,-


In [25]:
# rm overlapped with df_offs
df_offs = pd.concat(
    [
        df_old_offs.iloc[:,:3],
        df_new_offs.iloc[:,:3]
    ]
)
# df_offs
a = BedTool.from_dataframe(df_offs.iloc[:,:3])
b = BedTool.from_dataframe(df_tss.iloc[:,:3])
df_tss = b.intersect(a, loj=True).to_dataframe().query('score==-1').iloc[:, :3].merge(df_tss, on=['chrom', 'start', 'end'], how='left')
df_tss

,chrom,start,end,name,score,strand
0,chr1,63418,67418,NM_001005484,65418,+
1,chr1,449678,453678,NM_001005221,451678,-
2,chr1,684654,688654,NM_001005277,686654,-
3,chr1,921922,925922,NM_001385641,923922,+
4,chr1,923730,927730,NM_152486,925730,+
...,...,...,...,...,...,...
25976,chrY,24811393,24815393,NM_020364,24813393,-
25977,chrY,24831819,24835819,NM_020420,24833819,+
25978,chrY,24831918,24835918,NM_001388484,24833918,+
25979,chrY,25050104,25054104,NM_001002761,25052104,-


In [26]:
# with df_ctcf, without, df_ctcf
a = BedTool.from_dataframe(df_ctcf.iloc[:,:3])
b = BedTool.from_dataframe(df_tss.iloc[:,:3])
df_tss_has_ctcf = b.intersect(a, loj=True).to_dataframe().query('score!=-1').iloc[:, :3].merge(df_tss, on=['chrom', 'start', 'end'], how='left')
df_tss_no_ctcf = b.intersect(a, loj=True).to_dataframe().query('score==-1').iloc[:, :3].merge(df_tss, on=['chrom', 'start', 'end'], how='left')

In [27]:
df_tss_has_ctcf = df_tss_has_ctcf.drop_duplicates(keep='first')
df_tss_has_ctcf

,chrom,start,end,name,score,strand
0,chr1,921922,925922,NM_001385641,923922,+
3,chr1,923730,927730,NM_152486,925730,+
5,chr1,998097,1002097,NM_001142467,1000097,-
10,chr1,1018119,1022119,NM_198576,1020119,+
11,chr1,1114089,1118089,NM_001330306,1116089,-
...,...,...,...,...,...,...
15822,chrX,154749077,154753077,NM_001081573,154751077,-
15823,chrX,154760863,154764863,NM_001142463,154762863,+
15824,chrX,154803485,154807485,NM_001166462,154805485,-
15825,chrX,155262491,155266491,NM_171998,155264491,-


In [28]:
df_tss_no_ctcf = df_tss_no_ctcf.drop_duplicates(keep='first')
df_tss_no_ctcf

,chrom,start,end,name,score,strand
0,chr1,63418,67418,NM_001005484,65418,+
1,chr1,449678,453678,NM_001005221,451678,-
2,chr1,684654,688654,NM_001005277,686654,-
3,chr1,957256,961256,NM_015658,959256,-
4,chr1,958583,962583,NM_198317,960583,+
...,...,...,...,...,...,...
15033,chrY,24811393,24815393,NM_020364,24813393,-
15034,chrY,24831819,24835819,NM_020420,24833819,+
15035,chrY,24831918,24835918,NM_001388484,24833918,+
15036,chrY,25050104,25054104,NM_001002761,25052104,-


In [67]:
df_tss_has_ctcf.to_csv(f'../bed_for_count/2023-05-12_tss_has_ctcf_for_count.bed', header=False, index=False, sep='\t')
df_tss_no_ctcf.to_csv(f'../bed_for_count/2023-05-12_tss_no_ctcf_for_count.bed', header=False, index=False, sep='\t')

## 准备count-table

In [68]:
from glob import glob

In [69]:
# ls_bam = sorted(glob('../bam/ATACSeq_*.sortp_genome_rmchrM_rmdup.bam'))
# ls_bam
# dt_bam = {i.split('Seq_')[-1].split('.sortp')[0]: i for i in ls_bam}
# dt_bam

In [70]:
# ls_bed = sorted(glob('../bed_for_count/*.bed'))
# ls_bed
# dt_bed = {i.split('2023-05-12_')[-1].split('_for_count')[0]: i for i in ls_bed}
# dt_bed

In [71]:
# for bed_name, bed_path in dt_bed.items():
#     for bam_name, bam_path in dt_bam.items():
#         # print(bed_name)
#         pass
#         # print(bam_name)
#     #     count_table = f"../bed_for_count/AnalysisChanges_BAM_{bam_name}_BED_{bed_name}.count_table"
#     #     cmd = f"sleep 1\necho submit {count_table}\npkurun-cnlong 1 10 bedtools coverage -a {bed_path} -b {bam_path} > {count_table}\n"
#         # print(cmd)

In [72]:
ls_bam = sorted(glob('../bam/ATACSeq_*.sortp_genome_rmchrM_rmdup.bam'))
bams = [i.split('Seq_')[-1].split('.sortp')[0] for i in ls_bam]
ls_bed = sorted(glob('../bed_for_count/*.bed'))
beds = [i.split('2023-05-12_')[-1].split('_for_count')[0] for i in ls_bed]
print(bams)
print(beds)

['GFP-NLS_REP-1', 'GFP_REP-1', 'ND6-DddAwt_REP-1', 'ND6-DddAwt_REP-2', 'SIRT6-DddA11_REP-1', 'SIRT6-DddA11_REP-2']
['CTCF_medium', 'CTCF_strong', 'CTCF_weak', 'CTCF_within-offs', 'offs-ND6-DEP', 'offs-ND6-IND', 'random', 'tss_has_ctcf', 'tss_no_ctcf']


In [35]:
# # run snakemake
# BAMs = ['GFP-NLS_REP-1', 'GFP_REP-1', 'ND6-DddAwt_REP-1', 'ND6-DddAwt_REP-2', 'SIRT6-DddA11_REP-1', 'SIRT6-DddA11_REP-2']
# BEDs = ['CTCF_medium', 'CTCF_strong', 'CTCF_weak', 'CTCF_within-offs', 'offs-ND6-DEP', 'offs-ND6-IND', 'random']

## 分析比较ATAC-seq数据变化

In [73]:
from glob import glob
import pandas as pd

In [74]:
ls_counts = sorted(glob('../bed_for_count/AnalysisChanges_*.count_table'))
# ls_counts

In [75]:
df = pd.DataFrame(ls_counts, columns=['file_path'])
# df

In [76]:
df['sample'] = df.file_path.str.split('BAM_').str[-1].str.split('_BED').str[0]
df['region'] = df.file_path.str.split('BED_').str[-1].str.split('.count').str[0]
df

,file_path,sample,region
0,../bed_for_count/AnalysisChanges_BAM_GFP-NLS_R...,GFP-NLS_REP-1,CTCF_medium
1,../bed_for_count/AnalysisChanges_BAM_GFP-NLS_R...,GFP-NLS_REP-1,CTCF_strong
2,../bed_for_count/AnalysisChanges_BAM_GFP-NLS_R...,GFP-NLS_REP-1,CTCF_weak
3,../bed_for_count/AnalysisChanges_BAM_GFP-NLS_R...,GFP-NLS_REP-1,CTCF_within-offs
4,../bed_for_count/AnalysisChanges_BAM_GFP-NLS_R...,GFP-NLS_REP-1,offs-ND6-DEP
5,../bed_for_count/AnalysisChanges_BAM_GFP-NLS_R...,GFP-NLS_REP-1,offs-ND6-IND
6,../bed_for_count/AnalysisChanges_BAM_GFP-NLS_R...,GFP-NLS_REP-1,random
7,../bed_for_count/AnalysisChanges_BAM_GFP-NLS_R...,GFP-NLS_REP-1,tss_has_ctcf
8,../bed_for_count/AnalysisChanges_BAM_GFP-NLS_R...,GFP-NLS_REP-1,tss_no_ctcf
9,../bed_for_count/AnalysisChanges_BAM_GFP_REP-1...,GFP_REP-1,CTCF_medium


In [77]:
samples = sorted(df['sample'].unique())
regions = sorted(df['region'].unique())
regions
# samples

['CTCF_medium',
 'CTCF_strong',
 'CTCF_weak',
 'CTCF_within-offs',
 'offs-ND6-DEP',
 'offs-ND6-IND',
 'random',
 'tss_has_ctcf',
 'tss_no_ctcf']

In [78]:
df_total_reads = pd.read_csv('2023-05-12_step.03.MappingQC_result.csv', index_col=0).reset_index()
df_total_reads = df_total_reads[['index', 'map to genome remove duplicates (M)']].copy()
df_total_reads.columns = ['sample', 'total_reads_M']
df_total_reads = df_total_reads.query('sample!="test"').copy()
df_total_reads

,sample,total_reads_M
0,GFP-NLS_REP-1,28.781
1,GFP_REP-1,24.925
2,ND6-DddAwt_REP-1,24.676
3,ND6-DddAwt_REP-2,32.489
4,SIRT6-DddA11_REP-1,27.194
5,SIRT6-DddA11_REP-2,26.968


In [79]:
ls = []
for idx, line in df.iterrows():
    file_path, sample, region = line
    # print(file_path, sample, region)
    tmpdf = pd.read_csv(file_path, sep='\t', header=None)
    tmpdf['sample'] = sample
    tmpdf['region'] = region
    ls.append(tmpdf)
df_counts = pd.concat(ls)
df_counts.columns = ['chrom', 'start', 'end', 'name', 'score', 'strand', 'depth', '_1', '_2', '_3', 'sample', 'region']
df_counts = df_counts[['sample', 'region', 'chrom', 'start', 'end', 'name', 'score', 'strand', 'depth']].copy()
df_counts

,sample,region,chrom,start,end,name,score,strand,depth
0,GFP-NLS_REP-1,CTCF_medium,chr1,778863,779287,chr1_778863_779287_medium_peak,2.778652e+00,.,203
1,GFP-NLS_REP-1,CTCF_medium,chr1,919511,919935,chr1_919511_919935_medium_peak,2.369525e+00,.,18
2,GFP-NLS_REP-1,CTCF_medium,chr1,923010,923434,chr1_923010_923434_medium_peak,3.034356e+00,.,38
3,GFP-NLS_REP-1,CTCF_medium,chr1,939801,940225,chr1_939801_940225_medium_peak,2.608183e+00,.,69
4,GFP-NLS_REP-1,CTCF_medium,chr1,951368,951792,chr1_951368_951792_medium_peak,2.659323e+00,.,41
...,...,...,...,...,...,...,...,...,...
15033,SIRT6-DddA11_REP-2,tss_no_ctcf,chrY,24811393,24815393,NM_020364,2.481339e+07,-,0
15034,SIRT6-DddA11_REP-2,tss_no_ctcf,chrY,24831819,24835819,NM_020420,2.483382e+07,+,0
15035,SIRT6-DddA11_REP-2,tss_no_ctcf,chrY,24831918,24835918,NM_001388484,2.483392e+07,+,0
15036,SIRT6-DddA11_REP-2,tss_no_ctcf,chrY,25050104,25054104,NM_001002761,2.505210e+07,-,0


In [80]:
df_total_reads

,sample,total_reads_M
0,GFP-NLS_REP-1,28.781
1,GFP_REP-1,24.925
2,ND6-DddAwt_REP-1,24.676
3,ND6-DddAwt_REP-2,32.489
4,SIRT6-DddA11_REP-1,27.194
5,SIRT6-DddA11_REP-2,26.968


In [81]:
df_counts = df_counts.merge(df_total_reads, on='sample', how='left')
df_counts

,sample,region,chrom,start,end,name,score,strand,depth,total_reads_M
0,GFP-NLS_REP-1,CTCF_medium,chr1,778863,779287,chr1_778863_779287_medium_peak,2.778652e+00,.,203,28.781
1,GFP-NLS_REP-1,CTCF_medium,chr1,919511,919935,chr1_919511_919935_medium_peak,2.369525e+00,.,18,28.781
2,GFP-NLS_REP-1,CTCF_medium,chr1,923010,923434,chr1_923010_923434_medium_peak,3.034356e+00,.,38,28.781
3,GFP-NLS_REP-1,CTCF_medium,chr1,939801,940225,chr1_939801_940225_medium_peak,2.608183e+00,.,69,28.781
4,GFP-NLS_REP-1,CTCF_medium,chr1,951368,951792,chr1_951368_951792_medium_peak,2.659323e+00,.,41,28.781
...,...,...,...,...,...,...,...,...,...,...
570427,SIRT6-DddA11_REP-2,tss_no_ctcf,chrY,24811393,24815393,NM_020364,2.481339e+07,-,0,26.968
570428,SIRT6-DddA11_REP-2,tss_no_ctcf,chrY,24831819,24835819,NM_020420,2.483382e+07,+,0,26.968
570429,SIRT6-DddA11_REP-2,tss_no_ctcf,chrY,24831918,24835918,NM_001388484,2.483392e+07,+,0,26.968
570430,SIRT6-DddA11_REP-2,tss_no_ctcf,chrY,25050104,25054104,NM_001002761,2.505210e+07,-,0,26.968


In [82]:
df_counts['region_length'] = df_counts['end'] -  df_counts['start']
df_counts['RPKM'] = df_counts['depth'] / (df_counts['region_length']/1_000 * df_counts['total_reads_M'])
df_counts['region_index'] = df_counts['chrom'] + '_' + df_counts['start'].astype(str) + '_' + df_counts['end'].astype(str)
df_counts.drop_duplicates(subset=['sample', 'region', 'region_index'], keep='first', inplace=True)
df_counts.set_index(['region_index', 'region'], inplace=True)
df_counts

,,sample,chrom,start,end,name,score,strand,depth,total_reads_M,region_length,RPKM
region_index,region,,,,,,,,,,,
chr1_778863_779287,CTCF_medium,GFP-NLS_REP-1,chr1,778863,779287,chr1_778863_779287_medium_peak,2.778652e+00,.,203,28.781,424,16.635057
chr1_919511_919935,CTCF_medium,GFP-NLS_REP-1,chr1,919511,919935,chr1_919511_919935_medium_peak,2.369525e+00,.,18,28.781,424,1.475030
chr1_923010_923434,CTCF_medium,GFP-NLS_REP-1,chr1,923010,923434,chr1_923010_923434_medium_peak,3.034356e+00,.,38,28.781,424,3.113952
chr1_939801_940225,CTCF_medium,GFP-NLS_REP-1,chr1,939801,940225,chr1_939801_940225_medium_peak,2.608183e+00,.,69,28.781,424,5.654281
chr1_951368_951792,CTCF_medium,GFP-NLS_REP-1,chr1,951368,951792,chr1_951368_951792_medium_peak,2.659323e+00,.,41,28.781,424,3.359790
...,...,...,...,...,...,...,...,...,...,...,...,...
chrY_24811393_24815393,tss_no_ctcf,SIRT6-DddA11_REP-2,chrY,24811393,24815393,NM_020364,2.481339e+07,-,0,26.968,4000,0.000000
chrY_24831819_24835819,tss_no_ctcf,SIRT6-DddA11_REP-2,chrY,24831819,24835819,NM_020420,2.483382e+07,+,0,26.968,4000,0.000000
chrY_24831918_24835918,tss_no_ctcf,SIRT6-DddA11_REP-2,chrY,24831918,24835918,NM_001388484,2.483392e+07,+,0,26.968,4000,0.000000


In [83]:
df_counts.RPKM.describe()

count    568470.000000
mean          2.950206
std           6.291637
min           0.000000
25%           0.401204
50%           1.234090
75%           3.217199
max         387.884121
Name: RPKM, dtype: float64

In [84]:
import itertools as it
import math
from lets_plot import *
LetsPlot.setup_html()

In [87]:
ls = []
for sample_2 in it.combinations(samples, 2):
    s1, s2 = sample_2
    s_s1 = df_counts.query('sample==@s1')['RPKM']
    s_s2 = df_counts.query('sample==@s2')['RPKM']
    df_sample_2 = pd.concat([s_s1, s_s2], axis=1)
    df_sample_2.columns = ['RPKM1', 'RPKM2']
    df_sample_2 = df_sample_2[(df_sample_2.RPKM1 >= 0.5) & (df_sample_2.RPKM2 >= 0.5)].copy()
    df_sample_2['Log2_FC_RPKM'] = (df_sample_2['RPKM1'] / df_sample_2['RPKM2']).map(math.log10)
    df_sample_2['Geometric_mean'] = df_sample_2['RPKM1'].map(math.sqrt) * df_sample_2['RPKM2'].map(math.sqrt)
    df_sample_2 = df_sample_2[['Log2_FC_RPKM', 'Geometric_mean']].reset_index().copy()
    df_sample_2['compare'] = f'{s1} v.s. {s2}'
    ls.append(df_sample_2)
df_plot = pd.concat(ls)
df_plot

,region_index,region,Log2_FC_RPKM,Geometric_mean,compare
0,chr1_778863_779287,CTCF_medium,0.137815,14.194386,GFP-NLS_REP-1 v.s. GFP_REP-1
1,chr1_919511_919935,CTCF_medium,-0.284319,2.046259,GFP-NLS_REP-1 v.s. GFP_REP-1
2,chr1_923010_923434,CTCF_medium,0.025951,3.022291,GFP-NLS_REP-1 v.s. GFP_REP-1
3,chr1_939801_940225,CTCF_medium,0.060375,5.274604,GFP-NLS_REP-1 v.s. GFP_REP-1
4,chr1_951368_951792,CTCF_medium,0.103155,2.983559,GFP-NLS_REP-1 v.s. GFP_REP-1
...,...,...,...,...,...
60950,chrX_155378708_155382708,tss_no_ctcf,-0.075912,1.274168,SIRT6-DddA11_REP-1 v.s. SIRT6-DddA11_REP-2
60951,chrX_155380094_155384094,tss_no_ctcf,-0.090775,1.653367,SIRT6-DddA11_REP-1 v.s. SIRT6-DddA11_REP-2
60952,chrX_155456620_155460620,tss_no_ctcf,-0.036049,1.236179,SIRT6-DddA11_REP-1 v.s. SIRT6-DddA11_REP-2
60953,chrX_155458005_155462005,tss_no_ctcf,-0.060447,0.916587,SIRT6-DddA11_REP-1 v.s. SIRT6-DddA11_REP-2


In [88]:
print(df_plot.groupby(['compare', 'region']).region_index.count().to_frame().reset_index().to_string())

                                        compare            region  region_index
0                  GFP-NLS_REP-1 v.s. GFP_REP-1       CTCF_medium         11463
1                  GFP-NLS_REP-1 v.s. GFP_REP-1       CTCF_strong         12340
2                  GFP-NLS_REP-1 v.s. GFP_REP-1         CTCF_weak          9318
3                  GFP-NLS_REP-1 v.s. GFP_REP-1  CTCF_within-offs          1218
4                  GFP-NLS_REP-1 v.s. GFP_REP-1      offs-ND6-DEP            66
5                  GFP-NLS_REP-1 v.s. GFP_REP-1      offs-ND6-IND           526
6                  GFP-NLS_REP-1 v.s. GFP_REP-1            random          5182
7                  GFP-NLS_REP-1 v.s. GFP_REP-1      tss_has_ctcf         10617
8                  GFP-NLS_REP-1 v.s. GFP_REP-1       tss_no_ctcf          8051
9           GFP-NLS_REP-1 v.s. ND6-DddAwt_REP-1       CTCF_medium         11459
10          GFP-NLS_REP-1 v.s. ND6-DddAwt_REP-1       CTCF_strong         12337
11          GFP-NLS_REP-1 v.s. ND6-DddAw

In [51]:
df_plot.groupby('compare').Log2_FC_RPKM.describe()

,count,mean,std,min,25%,50%,75%,max
compare,,,,,,,,
GFP-NLS_REP-1 v.s. GFP_REP-1,58781.0,0.109124,0.161037,-0.965561,0.028449,0.113621,0.197723,1.051473
GFP-NLS_REP-1 v.s. ND6-DddAwt_REP-1,59186.0,0.147606,0.189976,-0.820159,0.042313,0.167252,0.271092,1.188441
GFP-NLS_REP-1 v.s. ND6-DddAwt_REP-2,59872.0,0.188708,0.187448,-0.947370,0.086054,0.213998,0.309306,1.228722
GFP-NLS_REP-1 v.s. SIRT6-DddA11_REP-1,60082.0,0.172706,0.197298,-0.869731,0.061848,0.199763,0.302726,1.168492
GFP-NLS_REP-1 v.s. SIRT6-DddA11_REP-2,59819.0,0.175846,0.197334,-0.873355,0.066299,0.200222,0.305820,1.175863
GFP_REP-1 v.s. ND6-DddAwt_REP-1,58698.0,0.040026,0.169678,-1.045753,-0.049111,0.058691,0.142573,0.898730
GFP_REP-1 v.s. ND6-DddAwt_REP-2,59276.0,0.082393,0.164931,-0.884899,-0.009838,0.103202,0.184548,0.939010
GFP_REP-1 v.s. SIRT6-DddA11_REP-1,59561.0,0.066935,0.176651,-0.916405,-0.032200,0.089425,0.179590,1.268287
GFP_REP-1 v.s. SIRT6-DddA11_REP-2,59348.0,0.069462,0.176351,-1.141878,-0.027934,0.092206,0.180342,1.210305


In [89]:
df_plot

,region_index,region,Log2_FC_RPKM,Geometric_mean,compare
0,chr1_778863_779287,CTCF_medium,0.137815,14.194386,GFP-NLS_REP-1 v.s. GFP_REP-1
1,chr1_919511_919935,CTCF_medium,-0.284319,2.046259,GFP-NLS_REP-1 v.s. GFP_REP-1
2,chr1_923010_923434,CTCF_medium,0.025951,3.022291,GFP-NLS_REP-1 v.s. GFP_REP-1
3,chr1_939801_940225,CTCF_medium,0.060375,5.274604,GFP-NLS_REP-1 v.s. GFP_REP-1
4,chr1_951368_951792,CTCF_medium,0.103155,2.983559,GFP-NLS_REP-1 v.s. GFP_REP-1
...,...,...,...,...,...
60950,chrX_155378708_155382708,tss_no_ctcf,-0.075912,1.274168,SIRT6-DddA11_REP-1 v.s. SIRT6-DddA11_REP-2
60951,chrX_155380094_155384094,tss_no_ctcf,-0.090775,1.653367,SIRT6-DddA11_REP-1 v.s. SIRT6-DddA11_REP-2
60952,chrX_155456620_155460620,tss_no_ctcf,-0.036049,1.236179,SIRT6-DddA11_REP-1 v.s. SIRT6-DddA11_REP-2
60953,chrX_155458005_155462005,tss_no_ctcf,-0.060447,0.916587,SIRT6-DddA11_REP-1 v.s. SIRT6-DddA11_REP-2


In [ ]:
g = (
    ggplot(data=df_plot, mapping=aes(x='Log2_FC_RPKM', y='Geometric_mean'))
    + geom_point()
    + geom_line(x=0, color='red')
    + facet_grid(x='compare', y='region', scales='free')
    + ggsize(width=5500, height=1500)
    + theme_minimal()
)
# g

In [ ]:
ggsave(g, filename='2023-05-12_ATAC-seq_LogFC_compare.html', path='.', iframe=True)

In [1]:
%matplotlib inline


# A simple PyDESeq2 workflow

In this example, we show how to perform a simple differential expression analysis on bulk
RNAseq data, using PyDESeq2.
    :depth: 3

We start by importing required packages and setting up an optional path to save results.


In [11]:
import os
import pickle as pkl
import pandas as pd

from pydeseq2.dds import DeseqDataSet
from pydeseq2.ds import DeseqStats
# from pydeseq2.utils import load_example_data

SAVE = False  # whether to save the outputs of this notebook

if SAVE:
    # Replace this with the path to directory where you would like results to be saved
    OUTPUT_PATH = "../output_files/synthetic_example"
    os.makedirs(OUTPUT_PATH, exist_ok=True)  # Create path if it doesn't exist

## Data loading

To perform differential expression analysis (DEA), PyDESeq2 requires two types of
inputs:

  * A count matrix of shape 'number of samples' x 'number of genes', containing
    read counts (non-negative integers),
  * Clinical data (or "column" data) of shape 'number of samples' x
    'number of variables', containing sample annotations that will be used
    to split the data in cohorts.

Both should be provided as [pandas dataframes](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html).

.. currentmodule:: pydeseq2

To illustrate the required data format, we load a synthetic example dataset that may be
obtained through PyDESeq2's API using :func:`utils.load_example_data`.
You may replace it with your own dataset.



In [47]:
# counts_df = load_example_data(
#     modality="raw_counts",
#     dataset="synthetic",
#     debug=True,
# )

# clinical_df = load_example_data(
#     modality="clinical",
#     dataset="synthetic",
#     debug=True,
# )
counts_df = pd.read_csv('test_counts.csv', index_col=0).T
clinical_df = pd.read_csv('test_clinical.csv', index_col=0)
print(counts_df)
print(clinical_df)

           gene1  gene2  gene3  gene4  gene5  gene6  gene7  gene8  gene9   
sample1       12     21      4    130     18      0     16     54     49  \
sample2        1     44      2     63     11     10     70     32     57   
sample3        4      4     11    180     21      3     28     34     65   
sample4        1     10      2    100     44      9     28     16     33   
sample5        1     11      6    135     16      2     32     29     31   
...          ...    ...    ...    ...    ...    ...    ...    ...    ...   
sample96       7     26      3     67     11      4     41     44     54   
sample97       1     14      3     71     33      5     19     42     25   
sample98      10     36      2     72     11      2     66     27     16   
sample99      18     14      3     66     53     11     32     19     79   
sample100     21      9      3     42     13     13     19     78     30   

           gene10  
sample1         3  
sample2         9  
sample3         2  
sample4

In this example, the clinical data contains two columns, ``condition`` and ``group``,
representing two types of bi-level annotations. In the first part, we will only use the
``condition`` factor. Later on, we'll see how to use both the `condition` and the
``group`` factors in our analysis (see `multifactor_ref`).



### Data filtering

Before proceeding with DEA, it is good practice to preprocess your data, e.g. to remove
samples for which annotations are missing and exclude genes with very low levels of
expression. This is not necessary in the case of our synthetic data, but don't forget
this step if you are using real data. To this end you can use the code below.

We start by removing samples for which ``condition`` is ``NaN``. If you are using
another dataset, do not forget to change "condition" for the column of ``clinical_df``
you wish to use as a design factor in your analysis.



In [49]:
samples_to_keep = ~clinical_df.condition.isna()
counts_df = counts_df.loc[samples_to_keep,:]
clinical_df = clinical_df.loc[samples_to_keep, :]

print(counts_df)
print(clinical_df)

           gene1  gene2  gene3  gene4  gene5  gene6  gene7  gene8  gene9   
sample1       12     21      4    130     18      0     16     54     49  \
sample2        1     44      2     63     11     10     70     32     57   
sample3        4      4     11    180     21      3     28     34     65   
sample4        1     10      2    100     44      9     28     16     33   
sample5        1     11      6    135     16      2     32     29     31   
...          ...    ...    ...    ...    ...    ...    ...    ...    ...   
sample96       7     26      3     67     11      4     41     44     54   
sample97       1     14      3     71     33      5     19     42     25   
sample98      10     36      2     72     11      2     66     27     16   
sample99      18     14      3     66     53     11     32     19     79   
sample100     21      9      3     42     13     13     19     78     30   

           gene10  
sample1         3  
sample2         9  
sample3         2  
sample4

<div class="alert alert-info"><h4>Note</h4><p>In the case where the design factor contains ``NaN`` entries, PyDESeq2 will throw an
  error when intializing a :class:`DeseqDataSet <dds.DeseqDataSet>`.</p></div>



Next, we filter out genes that have less than 10 read counts in total. Note again that
there are no such genes in this synthetic dataset.



In [50]:
print(counts_df.sum(axis=0))
genes_to_keep = counts_df.columns[counts_df.sum(axis=0) >= 10]
counts_df = counts_df[genes_to_keep]
counts_df

gene1       838
gene2      2259
gene3       485
gene4     10682
gene5      2870
gene6       545
gene7      2966
gene8      4277
gene9      3886
gene10     1170
dtype: int64


,gene1,gene2,gene3,gene4,gene5,gene6,gene7,gene8,gene9,gene10
sample1,12,21,4,130,18,0,16,54,49,3
sample2,1,44,2,63,11,10,70,32,57,9
sample3,4,4,11,180,21,3,28,34,65,2
sample4,1,10,2,100,44,9,28,16,33,9
sample5,1,11,6,135,16,2,32,29,31,5
...,...,...,...,...,...,...,...,...,...,...
sample96,7,26,3,67,11,4,41,44,54,1
sample97,1,14,3,71,33,5,19,42,25,4
sample98,10,36,2,72,11,2,66,27,16,9
sample99,18,14,3,66,53,11,32,19,79,11


Now that we have loaded and filtered our data, we may proceed with the differential
analysis.



## Single factor analysis

In this first analysis, we ignore the ``group`` variable and use the ``condition``
column as our design factor. That is, we compare gene expressions of samples that have
``condition B`` to those that have ``condition A``.




.. currentmodule:: pydeseq2.dds

### Read counts modeling with the :class:`DeseqDataSet` class

We start by creating a :class:`DeseqDataSet`
object from the count and clinical data.
A :class:`DeseqDataSet` fits dispersion and
log-fold change (LFC) parameters from the data, and stores them.




In [51]:
dds = DeseqDataSet(
    counts=counts_df,
    clinical=clinical_df,
    design_factors="condition",
    refit_cooks=True,
    n_cpus=8,
)

A :class:`DeseqDataSet` has two mandatory
arguments: a ``counts`` and a ``clinical`` dataframe, like the ones we've loaded in the
first part of this tutorial.

Next, we should specify the ``design_factor``, i.e. the column of the ``clinical``
dataframe that will be used to compare samples. This can be a single string as above,
or a list of strings, as in the
`section on multifactor analysis<multifactor_ref>`.

<div class="alert alert-info"><h4>Note</h4><p>The ``"condition"`` argument passed to ``design_factors`` corresponds to a column
  from the ``clinical_df`` dataframe we loaded earlier.
  You might need to change it according to your own dataset.</p></div>

Several other arguments may be optionally specified (see the :doc:`API documentation
</api/docstrings/pydeseq2.dds.DeseqDataSet>`).
Among those, the ``refit_cooks`` argument (set to ``True`` by default), controls
whether Cooks outlier should be refitted (which is advised, in general) and ``n_cpus``
sets the number of CPUs to use for computation. Here, we use 8 threads. Feel free to
adapt this to your setup or to set to ``None`` to use all available CPUs.

<div class="alert alert-info"><h4>Note</h4><p>In the case of the provided synthetic data, there won't be any Cooks outliers.</p></div>

Once a :class:`DeseqDataSet` was initialized,
we may run the :meth:`deseq2() <DeseqDataSet.deseq2>` method
to fit dispersions and LFCs.





In [52]:
dds.deseq2()


if SAVE:
    with open(os.path.join(OUTPUT_PATH, "dds.pkl"), "wb") as f:
        pkl.dump(dds, f)

Fitting size factors...
... done in 0.00 seconds.

Fitting dispersions...
... done in 0.01 seconds.

Fitting dispersion trend curve...
... done in 0.04 seconds.

Fitting MAP dispersions...
... done in 0.01 seconds.

Fitting LFCs...
... done in 0.01 seconds.

Refitting 0 outliers.



The :class:`DeseqDataSet` class extends the
:class:`AnnData <anndata.AnnData>`
class.



In [53]:
print(dds)

AnnData object with n_obs × n_vars = 100 × 10
    obs: 'condition', 'group'
    uns: 'trend_coeffs', '_squared_logres', 'prior_disp_var'
    obsm: 'design_matrix', 'size_factors', 'replaceable'
    varm: 'non_zero', '_rough_dispersions', 'genewise_dispersions', '_genewise_converged', '_normed_means', 'fitted_dispersions', 'MAP_dispersions', '_MAP_converged', 'dispersions', '_outlier_genes', 'LFC', '_LFC_converged', 'replaced'
    layers: 'normed_counts', '_mu_hat', '_mu_LFC', '_hat_diagonals', 'cooks'


Hence, parameters are stored according to the :class:`AnnData <anndata.AnnData>` data
structure, with key-based data fields. In particular,

- ``X`` stores the count data,
- ``obs`` stores design factors,
- ``obsm`` stores sample-level data, such as ``"design_matrix"`` and
  ``"size_factors"``,
- ``varm`` stores gene-level data, such as ``"dispersions"`` and ``"LFC"``.


As an example, here is how we would access dispersions and LFCs
(in natural log scale):



In [54]:
print(dds.varm["dispersions"])

[0.88259868 0.22257863 0.83723801 0.15897048 0.24992589 0.97364795
 0.23515487 0.19878078 0.1865203  0.63189989]


In [55]:
print(dds.varm["LFC"])

        intercept  condition_B_vs_A
gene1    1.891436          0.438632
gene2    2.851662          0.373296
gene3    1.787780         -0.438645
gene4    4.741958         -0.285647
gene5    3.077798          0.403457
gene6    1.678536          0.001010
gene7    3.291025          0.093116
gene8    3.785129         -0.187604
gene9    3.682882         -0.147443
gene10   2.300515          0.267562


.. currentmodule:: pydeseq2.ds

### Statistical analysis with the :class:`DeseqStats` class

Now that dispersions and LFCs were fitted, we may proceed with statistical tests to
compute p-values and adjusted p-values for differential expresion. This is the role of
the :class:`DeseqStats` class. It has a unique mandatory argument, ``dds``, which
should be a *fitted* :class:`DeseqDataSet <pydeseq2.dds.DeseqDataSet>`
object.



In [56]:
stat_res = DeseqStats(dds, n_cpus=8)

It also has a set of optional keyword arguments (see the :doc:`API documentation
</api/docstrings/pydeseq2.ds.DeseqStats>`), among which:

- ``alpha``: the p-value and adjusted p-value significance threshold (``0.05``
  by default),
- ``cooks_filter``: whether to filter p-values based on cooks outliers
  (``True`` by default),
- ``independent_filter``: whether to perform independent filtering to correct
  p-value trends (``True`` by default).

In the `section on multifactor analysis<multifactor_ref>`, we will also see how
to use the ``contrast`` argument to specify according to which variable samples should
be compared.




#### Wald test

PyDESeq2 computes p-values using Wald tests. This can be done using the
:meth:`summary() <DeseqStats.summary>` method, which runs the whole statistical
analysis, cooks filtering and multiple testing adjustement included.



In [57]:
stat_res.summary()

if SAVE:
    with open(os.path.join(OUTPUT_PATH, "stat_results.pkl"), "wb") as f:
        pkl.dump(stat_res, f)

Running Wald tests...
... done in 0.02 seconds.

Log2 fold change & Wald test p-value: condition B vs A


,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
gene1,8.541317,0.632812,0.289101,2.188898,0.028604,0.064150
gene2,21.281239,0.538552,0.149963,3.591235,0.000329,0.001646
gene3,5.010123,-0.632830,0.295236,-2.143475,0.032075,0.064150
gene4,100.517961,-0.412102,0.118629,-3.473867,0.000513,0.001710
gene5,27.142450,0.582065,0.154706,3.762408,0.000168,0.001646
gene6,5.413043,0.001457,0.310311,0.004696,0.996253,0.996253
gene7,28.294023,0.134338,0.149945,0.895917,0.370297,0.411441
gene8,40.358344,-0.270656,0.136401,-1.984260,0.047227,0.078711
gene9,37.166183,-0.212715,0.133243,-1.596437,0.110391,0.143148
gene10,11.589325,0.386011,0.244588,1.578207,0.114518,0.143148


The results are then stored in the ``results_df`` attribute (``stat_res.results_df``).



#### LFC shrinkage

For visualization or post-processing purposes, it might be suitable to perform
LFC shrinkage. This is implemented by the :meth:`lfc_shrink() <DeseqStats.lfc_shrink>`
method.



In [58]:
stat_res.lfc_shrink(coeff="condition_B_vs_A")

if SAVE:
    with open(os.path.join(OUTPUT_PATH, "shrunk_stat_results.pkl"), "wb") as f:
        pkl.dump(stat_res, f)

Fitting MAP LFCs...
... done in 0.02 seconds.

Shrunk Log2 fold change & Wald test p-value: condition B vs A


,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
gene1,8.541317,0.408253,0.294276,2.188898,0.028604,0.064150
gene2,21.281239,0.480145,0.151201,3.591235,0.000329,0.001646
gene3,5.010123,-0.396066,0.300796,-2.143475,0.032075,0.064150
gene4,100.517961,-0.374191,0.118704,-3.473867,0.000513,0.001710
gene5,27.142450,0.521487,0.156210,3.762408,0.000168,0.001646
gene6,5.413043,0.000716,0.239203,0.004696,0.996253,0.996253
gene7,28.294023,0.103421,0.141496,0.895917,0.370297,0.411441
gene8,40.358344,-0.226288,0.133477,-1.984260,0.047227,0.078711
gene9,37.166183,-0.175746,0.129138,-1.596437,0.110391,0.143148
gene10,11.589325,0.239935,0.231986,1.578207,0.114518,0.143148


<div class="alert alert-info"><h4>Note</h4><p>Running :meth:`lfc_shrink() <DeseqStats.lfc_shrink>` will overwrite a
  :class:`DeseqStats`' log fold changes (and standard errors) with shrunk values.
  This can be checked using the ``shrunk_LFCs`` flag.</p></div>




In [59]:
print(stat_res.shrunk_LFCs)  # Will be True only if lfc_shrink() was run.

True



## Multifactor analysis

.. currentmodule:: pydeseq2.dds

So far, we have only used the ``condition`` column of ``clinical_df``, which divides
samples between conditions ``A`` and ``B``. Yet, ``clinical_df`` contains second
column, which separates samples according to ``group`` ``X`` and ``Y``.



In [60]:
print(clinical_df)

          condition group
sample1           A     X
sample2           A     Y
sample3           A     X
sample4           A     Y
sample5           A     X
...             ...   ...
sample96          B     Y
sample97          B     X
sample98          B     Y
sample99          B     X
sample100         B     Y

[100 rows x 2 columns]


The goal of multifactor analysis is to use *both* variables to fit LFCs.

### Read counts modeling

To perform multifactor analysis with PyDESeq2, we start by inializing a
:class:`DeseqDataSet` as previously, but we provide the list of variables we would like
to use in the ``design_factors`` argument.




In [61]:
dds = DeseqDataSet(
    counts=counts_df,
    clinical=clinical_df,
    design_factors=["group", "condition"],
    refit_cooks=True,
    n_cpus=8,
)

<div class="alert alert-info"><h4>Note</h4><p>By default, the last variable in the list (here, ``"condition"``) will be the one for
  which LFCs and p-values will be displayed, but this may be changed later on when
  performing the statistical analysis.</p></div>

As for the single-factor analysis, we fit dispersions and LFCs using the
:meth:`deseq2() <DeseqDataSet.deseq2>` method.



In [62]:
dds.deseq2()

Fitting size factors...
... done in 0.00 seconds.

Fitting dispersions...
... done in 0.01 seconds.

Fitting dispersion trend curve...
... done in 0.03 seconds.

Fitting MAP dispersions...
... done in 0.01 seconds.

Fitting LFCs...
... done in 0.01 seconds.

Refitting 0 outliers.



Now, if we print log fold changes, we will have two columns in addition to the
intercept: one corresponding to the ``group`` variable, and the other to ``condition``.



In [63]:
print(dds.varm["LFC"])

        intercept  group_Y_vs_X  condition_B_vs_A
gene1    1.560313      0.525572          0.507013
gene2    2.812255      0.079229          0.371026
gene3    2.059577     -0.602606         -0.467002
gene4    4.919837     -0.385197         -0.293527
gene5    2.973802      0.194078          0.407582
gene6    1.846061     -0.352410         -0.013510
gene7    3.235967      0.106773          0.093654
gene8    3.640464      0.270819         -0.188248
gene9    3.645334      0.074474         -0.148346
gene10   2.136670      0.303009          0.270003


.. currentmodule:: pydeseq2.ds

### Statistical analysis

P-values are computed as earlier from a :class:`DeseqStats` object with the
:meth:`summary() <DeseqStats.summary>` method, with a new important argument:
the ``contrast``.
It is a list of three strings of the form
``["variable", "tested level", "reference level"]`` which determines which
variable we want to compute LFCs and pvalues for.
As an example, to compare the condition B to the condition A, we set
``contrast=["condition", "B", "A"]``.




In [64]:
stat_res_B_vs_A = DeseqStats(dds, contrast=["condition", "B", "A"], n_cpus=8)

<div class="alert alert-info"><h4>Note</h4><p>If left blank, the variable of interest will be the last one provided in
  the ``design_factors`` attribute of the corresponding
  :class:`DeseqDataSet <pydeseq2.dds.DeseqDataSet>` object,
  and the reference level will be picked alphabetically.
  In any case, *both variables are still used*. This is due to the fact that ``dds``
  was fit with both as design factors.</p></div>

Let us fit p-values:




In [65]:
stat_res_B_vs_A.summary()

Running Wald tests...
... done in 0.01 seconds.

Log2 fold change & Wald test p-value: condition B vs A


,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
gene1,8.541317,0.731466,0.286275,2.555117,0.010615,0.026538
gene2,21.281239,0.535278,0.149824,3.572711,0.000353,0.001178
gene3,5.010123,-0.673742,0.287400,-2.344268,0.019064,0.038129
gene4,100.517961,-0.423471,0.106221,-3.986712,0.000067,0.000592
gene5,27.142450,0.588016,0.152758,3.849332,0.000118,0.000592
gene6,5.413043,-0.019490,0.307845,-0.063313,0.949518,0.949518
gene7,28.294023,0.135114,0.149561,0.903410,0.366308,0.407009
gene8,40.358344,-0.271584,0.131512,-2.065084,0.038915,0.064858
gene9,37.166183,-0.214018,0.132989,-1.609288,0.107553,0.139684
gene10,11.589325,0.389532,0.244929,1.590388,0.111747,0.139684


As we can see, although we are comparing the same cohorts (condition B vs A), the
results differ from the `single-factor analysis <wald_ref>`. This is because the
model uses information from both the ``condition`` and ``group`` variables.

Let us now evaluate differential expression according to group Y vs X. To do so,
we create a new :class:`DeseqStats` from the same
:class:`DeseqDataSet <pydeseq2.dds.DeseqDataSet>`
with ``contrast=["group", "Y", "X"]``, and run the analysis again.



In [66]:
stat_res_Y_vs_X = DeseqStats(dds, contrast=["group", "Y", "X"], n_cpus=8)
stat_res_Y_vs_X.summary()

Running Wald tests...
... done in 0.01 seconds.

Log2 fold change & Wald test p-value: group Y vs X


,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
gene1,8.541317,0.758241,0.286328,2.648155,8.093232e-03,0.020233
gene2,21.281239,0.114304,0.149771,0.763189,4.453508e-01,0.445351
gene3,5.010123,-0.869376,0.287516,-3.023748,2.496646e-03,0.009903
gene4,100.517961,-0.555721,0.106221,-5.231722,1.679380e-07,0.000002
gene5,27.142450,0.279995,0.152731,1.833256,6.676450e-02,0.123874
gene6,5.413043,-0.508421,0.307951,-1.650980,9.874262e-02,0.141061
gene7,28.294023,0.154041,0.149565,1.029927,3.030443e-01,0.378805
gene8,40.358344,0.390709,0.131519,2.970732,2.970912e-03,0.009903
gene9,37.166183,0.107443,0.132982,0.807956,4.191157e-01,0.445351
gene10,11.589325,0.437149,0.244955,1.784611,7.432438e-02,0.123874


#### LFC shrinkage (multifactor)

In a multifactor setting, LFC shrinkage works as in the single-factor case, but will
only shrink the LFCs of a :class:`DeseqStats` object based on its
``contrast`` argument.



In [67]:
stat_res_B_vs_A.lfc_shrink(coeff="condition_B_vs_A")

Fitting MAP LFCs...
... done in 0.02 seconds.

Shrunk Log2 fold change & Wald test p-value: condition B vs A


,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
gene1,8.541317,0.526936,0.299297,2.555117,0.010615,0.026538
gene2,21.281239,0.479377,0.150834,3.572711,0.000353,0.001178
gene3,5.010123,-0.463492,0.296136,-2.344268,0.019064,0.038129
gene4,100.517961,-0.394989,0.106316,-3.986712,0.000067,0.000592
gene5,27.142450,0.531655,0.154062,3.849332,0.000118,0.000592
gene6,5.413043,-0.008636,0.241570,-0.063313,0.949518,0.949518
gene7,28.294023,0.106695,0.141879,0.903410,0.366308,0.407009
gene8,40.358344,-0.233346,0.129070,-2.065084,0.038915,0.064858
gene9,37.166183,-0.180064,0.129246,-1.609288,0.107553,0.139684
gene10,11.589325,0.251471,0.233140,1.590388,0.111747,0.139684


In [1]:
%matplotlib inline


# Step-by-step PyDESeq2 workflow

This notebook details all the steps of the PyDESeq2 pipeline.

It allows you to run the PyDESeq2 pipeline on the synthetic data provided
in this repository.

If this is your first contact with PyDESeq2, we recommend you first have a look at the
:doc:`standard workflow example <plot_minimal_pydeseq2_pipeline>`.
    :depth: 3

We start by importing required packages and setting up an optional path to save
results.


In [4]:
import os
import pickle as pkl
import pandas as pd

from pydeseq2.dds import DeseqDataSet
from pydeseq2.ds import DeseqStats
# from pydeseq2.utils import load_example_data

SAVE = False  # whether to save the outputs of this notebook

if SAVE:
    # Replace this with the path to directory where you would like results to be
    # saved
    OUTPUT_PATH = "../output_files/synthetic_example"
    os.makedirs(OUTPUT_PATH, exist_ok=True)  # Create path if it doesn't exist

## Data loading

Note that we are also explaining this step in the 'getting started' example.
To perform differential expression analysis (DEA), PyDESeq2 requires two types of
inputs:

  * A count matrix of shape 'number of samples' x 'number of genes', containing
    read counts (non-negative integers),
  * Clinical data (or "column" data) of shape 'number of samples' x
    'number of variables', containing sample annotations that will be used
    to split the data in cohorts.

Both should be provided as [pandas dataframes](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html).

.. currentmodule:: pydeseq2

To illustrate the required data format, we load a synthetic example dataset
that may be
obtained through PyDESeq2's API using :func:`utils.load_example_data`.
You may replace it with your own dataset.



In [5]:
# counts_df = load_example_data(
#     modality="raw_counts",
#     dataset="synthetic",
#     debug=True,
# )

# clinical_df = load_example_data(
#     modality="clinical",
#     dataset="synthetic",
#     debug=True,
# )
counts_df = pd.read_csv('test_counts.csv', index_col=0).T
clinical_df = pd.read_csv('test_clinical.csv', index_col=0)
print(counts_df)
print(clinical_df)

           gene1  gene2  gene3  gene4  gene5  gene6  gene7  gene8  gene9   
sample1       12     21      4    130     18      0     16     54     49  \
sample2        1     44      2     63     11     10     70     32     57   
sample3        4      4     11    180     21      3     28     34     65   
sample4        1     10      2    100     44      9     28     16     33   
sample5        1     11      6    135     16      2     32     29     31   
...          ...    ...    ...    ...    ...    ...    ...    ...    ...   
sample96       7     26      3     67     11      4     41     44     54   
sample97       1     14      3     71     33      5     19     42     25   
sample98      10     36      2     72     11      2     66     27     16   
sample99      18     14      3     66     53     11     32     19     79   
sample100     21      9      3     42     13     13     19     78     30   

           gene10  
sample1         3  
sample2         9  
sample3         2  
sample4

## 1. Read counts modeling
Read counts modeling with the :class:`DeseqDataSet
<dds.DeseqDataSet>` class

The :class:`DeseqDataSet <dds.DeseqDataSet>` class has two mandatory
arguments, `counts_df` and
`clinical_df`, as well as a set of optional keyword arguments, among which:

- `design_factor`: the name of the column of clinical to be used as a design
  variable
- `refit_cooks`: whether to refit cooks outliers – this is advised, in general.

<div class="alert alert-info"><h4>Note</h4><p>in the case of the provided synthetic data, there won't be any Cooks
  outliers.</p></div>



In [6]:
dds = DeseqDataSet(
    counts=counts_df,
    clinical=clinical_df,
    design_factors="condition",  # compare samples based on the "condition"
    # column ("B" vs "A")
    refit_cooks=True,
    n_cpus=8,
)

### Compute normalization factors



In [7]:
dds.fit_size_factors()

dds.obsm["size_factors"]

Fitting size factors...
... done in 0.04 seconds.



array([1.22898094, 1.18877375, 0.99722229, 1.00215773, 0.83457743,
       1.10730382, 0.8999001 , 1.15343785, 0.68163849, 1.29764537,
       1.04491511, 1.45930946, 1.14588441, 0.8049275 , 0.88402672,
       0.88402672, 1.32879767, 0.82564657, 1.5978062 , 1.29764537,
       1.31940196, 0.69919197, 1.10697146, 1.10214803, 1.19152118,
       1.0624452 , 0.98548229, 0.76881428, 0.8939601 , 1.27135863,
       1.61101905, 1.55084302, 0.83601298, 0.98213727, 1.27270212,
       1.0510719 , 1.76078144, 1.08132885, 1.50390106, 1.0510719 ,
       0.80280751, 0.70955247, 1.32602392, 0.98031899, 1.1078077 ,
       0.68792508, 0.90429564, 1.56411155, 0.81918767, 1.19364837,
       0.79492024, 1.84963565, 0.79694628, 0.79708276, 0.97287297,
       1.16248554, 1.50489413, 1.41929759, 1.04612122, 1.05720226,
       0.99635345, 1.84224912, 1.03801163, 0.89633874, 0.72952979,
       1.33453944, 0.93968061, 1.14016425, 1.59166589, 1.08554239,
       0.72370261, 0.91558563, 1.14183629, 1.33857618, 0.94450

### Fit genewise dispersions



In [8]:
dds.fit_genewise_dispersions()

dds.varm["genewise_dispersions"]

Fitting dispersions...
... done in 0.13 seconds.



array([0.91013511, 0.21371805, 0.81359443, 0.1613621 , 0.24850424,
       0.97307734, 0.23303023, 0.19810261, 0.18363604, 0.64637486])

### Fit dispersion trend coefficients



In [9]:
dds.fit_dispersion_trend()
dds.uns["trend_coeffs"]
dds.varm["fitted_dispersions"]

Fitting dispersion trend curve...
... done in 0.38 seconds.



array([0.65142791, 0.31300248, 1.04987097, 0.13414632, 0.26400662,
       0.97813348, 0.25676617, 0.20575176, 0.2160277 , 0.50274843])

### Dispersion priors



In [10]:
dds.fit_dispersion_prior()
print(
    f"logres_prior={dds.uns['_squared_logres']}, sigma_prior={dds.uns['prior_disp_var']}"
)

logres_prior=0.055924632043151165, sigma_prior=0.25


### MAP Dispersions
The `fit_MAP_dispersions` method filters the genes for which dispersion
shrinkage is applied.
Indeed, for genes whose MLE dispersions are too high above the trend curve,
the original MLE value is kept.
The final values of the dispersions that are used for downstream analysis is
stored in `dds.dispersions`.



In [11]:
dds.fit_MAP_dispersions()
dds.varm["MAP_dispersions"]
dds.varm["dispersions"]

Fitting MAP dispersions...
... done in 0.05 seconds.



array([0.88259868, 0.22257863, 0.83723801, 0.15897048, 0.24992589,
       0.97364795, 0.23515487, 0.19878078, 0.1865203 , 0.63189989])

### Fit log fold changes
Note that in the `DeseqDataSet` object, the log-fold changes are stored in
natural
log scale, but that the results dataframe output by the `summary` method of
`DeseqStats` displays LFCs in log2 scale (see later on).



In [12]:
dds.fit_LFC()
dds.varm["LFC"]

Fitting LFCs...
... done in 0.45 seconds.



,intercept,condition_B_vs_A
gene1,1.891436,0.438632
gene2,2.851662,0.373296
gene3,1.787780,-0.438645
gene4,4.741958,-0.285647
gene5,3.077798,0.403457
gene6,1.678536,0.001010
gene7,3.291025,0.093116
gene8,3.785129,-0.187604
gene9,3.682882,-0.147443
gene10,2.300515,0.267562


### Calculate Cooks distances and refit
Note that this step is optional.



In [13]:
dds.calculate_cooks()
if dds.refit_cooks:
    # Replace outlier counts
    dds.refit()

Refitting 0 outliers.



Save everything



In [14]:
if SAVE:
    with open(os.path.join(OUTPUT_PATH, "dds_detailed_pipe.pkl"), "wb") as f:
        pkl.dump(dds, f)

## 2. Statistical analysis
Statistical analysis with the :class:`DeseqStats <ds.DeseqStats>` class.
The `DeseqDataSet` class has a unique mandatory arguments, `dds`, which should
be a *fitted* `DeseqDataSet` object, as well as a set of optional keyword
arguments, among which:

- `alpha`: the p-value and adjusted p-value significance threshold
- `cooks_filter`: whether to filter p-values based on cooks outliers
- `independent_filter`: whether to perform independent filtering to correct
  p-value trends.



In [15]:
stat_res = DeseqStats(dds, alpha=0.05, cooks_filter=True, independent_filter=True)

### Wald tests



In [16]:
stat_res.run_wald_test()
stat_res.p_values

Running Wald tests...
... done in 0.29 seconds.



gene1     0.028604
gene2     0.000329
gene3     0.032075
gene4     0.000513
gene5     0.000168
gene6     0.996253
gene7     0.370297
gene8     0.047227
gene9     0.110391
gene10    0.114518
dtype: float64

### Cooks filtering
This is optional

<div class="alert alert-info"><h4>Note</h4><p>Note: in the case of the provided synthetic data, there won't be any
  outliers.</p></div>



In [17]:
if stat_res.cooks_filter:
    stat_res._cooks_filtering()
stat_res.p_values

gene1     0.028604
gene2     0.000329
gene3     0.032075
gene4     0.000513
gene5     0.000168
gene6     0.996253
gene7     0.370297
gene8     0.047227
gene9     0.110391
gene10    0.114518
dtype: float64

### P-value adjustment



In [18]:
if stat_res.independent_filter:
    stat_res._independent_filtering()
else:
    stat_res._p_value_adjustment()

stat_res.padj

gene1     0.064150
gene2     0.001646
gene3     0.064150
gene4     0.001710
gene5     0.001646
gene6     0.996253
gene7     0.411441
gene8     0.078711
gene9     0.143148
gene10    0.143148
Name: 0, dtype: float64

### Building a results dataframe
This dataframe is stored in the `results_df` attribute of the `DeseqStats`
class.



In [19]:
stat_res.summary()

Log2 fold change & Wald test p-value: condition B vs A


,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
gene1,8.541317,0.632812,0.289101,2.188898,0.028604,0.064150
gene2,21.281239,0.538552,0.149963,3.591235,0.000329,0.001646
gene3,5.010123,-0.632830,0.295236,-2.143475,0.032075,0.064150
gene4,100.517961,-0.412102,0.118629,-3.473867,0.000513,0.001710
gene5,27.142450,0.582065,0.154706,3.762408,0.000168,0.001646
gene6,5.413043,0.001457,0.310311,0.004696,0.996253,0.996253
gene7,28.294023,0.134338,0.149945,0.895917,0.370297,0.411441
gene8,40.358344,-0.270656,0.136401,-1.984260,0.047227,0.078711
gene9,37.166183,-0.212715,0.133243,-1.596437,0.110391,0.143148
gene10,11.589325,0.386011,0.244588,1.578207,0.114518,0.143148


Save everything if SAVE is set to True



In [20]:
if SAVE:
    with open(os.path.join(OUTPUT_PATH, "stat_results_detailed_pipe.pkl"), "wb") as f:
        pkl.dump(stat_res, f)

### LFC Shrinkage
For visualization or post-processing purposes, it might be suitable to perform
LFC shrinkage. This is implemented by the `lfc_shrink` method.



In [21]:
stat_res.lfc_shrink(coeff="condition_B_vs_A")

Fitting MAP LFCs...
... done in 2.29 seconds.

Shrunk Log2 fold change & Wald test p-value: condition B vs A


,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
gene1,8.541317,0.408253,0.294276,2.188898,0.028604,0.064150
gene2,21.281239,0.480145,0.151201,3.591235,0.000329,0.001646
gene3,5.010123,-0.396066,0.300796,-2.143475,0.032075,0.064150
gene4,100.517961,-0.374191,0.118704,-3.473867,0.000513,0.001710
gene5,27.142450,0.521487,0.156210,3.762408,0.000168,0.001646
gene6,5.413043,0.000716,0.239203,0.004696,0.996253,0.996253
gene7,28.294023,0.103421,0.141496,0.895917,0.370297,0.411441
gene8,40.358344,-0.226288,0.133477,-1.984260,0.047227,0.078711
gene9,37.166183,-0.175746,0.129138,-1.596437,0.110391,0.143148
gene10,11.589325,0.239935,0.231986,1.578207,0.114518,0.143148


Save everything



In [22]:
if SAVE:
    with open(
        os.path.join(OUTPUT_PATH, "shrunk_stat_results_detailed_pipe.pkl"), "wb"
    ) as f:
        pkl.dump(stat_res, f)